## Credit Approval Application.

Banks give loans to applicants.....


Plans and process
- Load dataset and view 
- analyse the column types and values (numerical or non numberical)
- Preprocess the dataset to ensure the ML model we choose can make good predictions
- Do some data analysis to build our instuition
- Build a machine learning model that cna predict if an individuals Credit Application will be approved or not 

In [8]:
import pandas as pd
import numpy as np
import matplotlib as plt 



In [6]:
cc_apps = pd.read_csv("D:/A work/DS/Credit_card_approval/Data-Science-Credit-Card-Approvals-Quant/datasets/cc_approvals.data", header= None)

### 1. Inspecting the data

In [7]:
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [9]:
cc_apps.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], dtype='int64')

The data has been anonymized to protect the privacy but this post here (http://rstudio-pubs-static.s3.amazonaws.com/73039_9946de135c0a49daa7a0a9eda4a67a72.html) explains the data. From the post we se that we have the following columns:
- Gender, Age, Debt, Married, BankCustomer, EducationLevel, Ethnicity, YearsEmployed, PriorDefault, Employed, CreditScore, DriversLicense, Citizen, ZipCode, Income and finally the ApprovalStatus



In [16]:
cc_apps_description = cc_apps.describe()
print(cc_apps_description)

print("\n")

cc_apps_info = cc_apps.info()
print(cc_apps_info)


               2           7          10             14
count  690.000000  690.000000  690.00000     690.000000
mean     4.758725    2.223406    2.40000    1017.385507
std      4.978163    3.346513    4.86294    5210.102598
min      0.000000    0.000000    0.00000       0.000000
25%      1.000000    0.165000    0.00000       0.000000
50%      2.750000    1.000000    0.00000       5.000000
75%      7.207500    2.625000    3.00000     395.500000
max     28.000000   28.500000   67.00000  100000.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
0     690 non-null object
1     690 non-null object
2     690 non-null float64
3     690 non-null object
4     690 non-null object
5     690 non-null object
6     690 non-null object
7     690 non-null float64
8     690 non-null object
9     690 non-null object
10    690 non-null int64
11    690 non-null object
12    690 non-null object
13    690 non-null object
14    690 non-null int64

In [17]:
# Renaming the columns to make the process readable 
cc_apps_copy = cc_apps.copy()

cc_apps_copy.columns = [
    'gender','age','debt','married','bankCustomer','educationalLevel','ethinicy','yearsEmployed','priorDefault','employed','creditScore','driversLicense','citizen','zipCode','income','approved']

cc_apps_copy.head()

,gender,age,debt,married,bankCustomer,educationalLevel,ethinicy,yearsEmployed,priorDefault,employed,creditScore,driversLicense,citizen,zipCode,income,approved
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


### 2. Handling missing values

- The dataset contains both numeric and non-numeric data (specifically data that are of float64, int64 and object types). Specifically, the features debt, yearsEmployed, creditScore and income contain numeric values (of types float64, float64, int64 and int64 respectively) and all the other features contain non-numeric values.
- The dataset also contains values from several ranges. Some features have a value range of 0 - 28, some have a range of 2 - 67, and some have a range of 1017 - 100000. Apart from these, we can get useful statistical information (like mean, max, and min) about the features that have numerical values.
- Finally, the dataset has missing values. The missing values in the dataset are labeled with '?'



In [23]:
cc_apps_copy.tail(10)

,gender,age,debt,married,bankCustomer,educationalLevel,ethinicy,yearsEmployed,priorDefault,employed,creditScore,driversLicense,citizen,zipCode,income,approved
680,b,19.50,0.290,u,g,k,v,0.290,f,f,0,f,g,00280,364,-
681,b,27.83,1.000,y,p,d,h,3.000,f,f,0,f,g,00176,537,-
682,b,17.08,3.290,u,g,i,v,0.335,f,f,0,t,g,00140,2,-
683,b,36.42,0.750,y,p,d,v,0.585,f,f,0,f,g,00240,3,-
684,b,40.58,3.290,u,g,m,v,3.500,f,f,0,t,s,00400,0,-
685,b,21.08,10.085,y,p,e,h,1.250,f,f,0,f,g,00260,0,-
686,a,22.67,0.750,u,g,c,v,2.000,f,t,2,t,g,00200,394,-
687,a,25.25,13.500,y,p,ff,ff,2.000,f,t,1,t,g,00200,1,-
688,b,17.92,0.205,u,g,aa,v,0.040,f,f,0,f,g,00280,750,-
689,b,35.00,3.375,u,g,c,h,8.290,f,f,0,t,g,00000,0,-


In [29]:
for column in cc_apps_copy.columns:
    print(column)
    filter = cc_apps_copy[column] == "?"

gender
age
debt


TypeError: invalid type comparison

In [35]:
import numpy as np


print(cc_apps_copy.isnull().values.sum())

# replace ? with NaN
cc_apps_copy = cc_apps_copy.replace('?', np.nan)

#inspect the missing values again
print('Toatal Nan: ' + str(cc_apps_copy.isnull().values.sum()))
print('NaN by column:' '\n')  
print(cc_apps.isnull().sum())
cc_apps_copy.tail(20)

67
Toatal Nan: 67
NaN by column:

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
dtype: int64


,gender,age,debt,married,bankCustomer,educationalLevel,ethinicy,yearsEmployed,priorDefault,employed,creditScore,driversLicense,citizen,zipCode,income,approved
670,b,47.17,5.835,u,g,w,v,5.500,f,f,0,f,g,00465,150,-
671,b,25.83,12.835,u,g,cc,v,0.500,f,f,0,f,g,00000,2,-
672,a,50.25,0.835,u,g,aa,v,0.500,f,f,0,t,g,00240,117,-
673,NaN,29.50,2.000,y,p,e,h,2.000,f,f,0,f,g,00256,17,-
674,a,37.33,2.500,u,g,i,h,0.210,f,f,0,f,g,00260,246,-
675,a,41.58,1.040,u,g,aa,v,0.665,f,f,0,f,g,00240,237,-
676,a,30.58,10.665,u,g,q,h,0.085,f,t,12,t,g,00129,3,-
677,b,19.42,7.250,u,g,m,v,0.040,f,t,1,f,g,00100,1,-
678,a,17.92,10.210,u,g,ff,ff,0.000,f,f,0,f,g,00000,50,-
679,a,20.08,1.250,u,g,c,v,0.000,f,f,0,f,g,00000,0,-


#### Handling missing values for numeric columns.

It is important to handle missing values before we apply any models. Dropping them may lead to loss of information and we don't want that. One common way to handle missing values is to input the mean values - mean imputation

In [36]:
cc_apps_copy.fillna(cc_apps_copy.mean(), inplace=True)

# Count the number of NaNs in the dataset to verify
print('Total NaN: ' + str(cc_apps.isnull().values.sum()))
cc_apps.isnull().sum()

Total NaN: 0


0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
dtype: int64

#### Handling missing values for non-numeric columns.

We are going to impute these missing values with the most frequent values as present in the respective columns. This is good practice when it comes to imputing missing values for categorical data in general.

In [39]:
for col in cc_apps_copy:
    # check if the column type is 'object
    if cc_apps_copy[col].dtypes == 'object':
        # impute the mode value
        cc_apps_copy = cc_apps_copy.fillna(cc_apps_copy[col].value_counts().index[0])

# Count the number of NaNs in the dataset and print the counts to verify
print('Total missing values:' + str(cc_apps_copy.isnull().values.sum()))
print('Missing values in each column:')
cc_apps_copy.isnull().sum()

Total missing values:0
Missing values in each column:


gender              0
age                 0
debt                0
married             0
bankCustomer        0
educationalLevel    0
ethinicy            0
yearsEmployed       0
priorDefault        0
employed            0
creditScore         0
driversLicense      0
citizen             0
zipCode             0
income              0
approved            0
dtype: int64